In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np

In [61]:
mp3_head = 'https://www.redkaraoke.com/data/mp3/es/'
mp3_aws_head = 'https://s3.amazonaws.com/redkaraoke/data/mp3/'
mp4_aws_head = 'https://s3.amazonaws.com/redkaraokecom/data/vid/'

In [46]:
links_list = []
for i in range(100):
    r = requests.get('https://www.redkaraoke.com/karaoke/celine-dion/my-heart-will-go-on/11827/recordings/p' + str(i))
    soup = BeautifulSoup(r.content, 'html.parser')
    for link in soup.find_all('a', attrs = {'class':'result result-recording'}):
        links_list.append('https://www.redkaraoke.com' + link.get('href'))
# Remove duplicated links
links_list_unique = set(links_list)
links_list_unique

{'https://www.redkaraoke.com/profile/Allira93/recordings/343699',
 'https://www.redkaraoke.com/profile/AngelJamae/recordings/664612',
 'https://www.redkaraoke.com/profile/Angelsoft/recordings/325316',
 'https://www.redkaraoke.com/profile/Angelsoft/recordings/342056',
 'https://www.redkaraoke.com/profile/Angelsoft/recordings/342084',
 'https://www.redkaraoke.com/profile/Angelsoft/recordings/356458',
 'https://www.redkaraoke.com/profile/Angelsoft/recordings/368931',
 'https://www.redkaraoke.com/profile/Angelsoft/recordings/386275',
 'https://www.redkaraoke.com/profile/Anti_Suwaryo/recordings/728780',
 'https://www.redkaraoke.com/profile/Arena55/recordings/648309',
 'https://www.redkaraoke.com/profile/BERNITA/recordings/1305787',
 'https://www.redkaraoke.com/profile/Bea18/recordings/624790',
 'https://www.redkaraoke.com/profile/CallieFord/recordings/65921',
 'https://www.redkaraoke.com/profile/CarmenDeMaria/recordings/1927954',
 'https://www.redkaraoke.com/profile/EnioPedrozo/recordings/3

In [54]:
print(len(links_list))
print(len(links_list_unique))

2400
122


In [48]:
# Features in javascript object "userRecording" :
# features = ['file','title','u','uavatar','ugroup','a','rid','adv']
# features1 = ['no_of_views','no_of_likes','no_of_comments','no_of_recordings','no_of_followers','no_of_following','date_recorded','gender_loc']
# features2 = ['recording_id','user_id','user_name','recording_url','file_url_tail','user_group']

counter = 0

#Initialize lists
view_list = []
like_list = []
comment_list = []
recording_list = []
follower_list = []
following_list = []
date_list = []
gender_loc_list = []

recording_id_list = []
user_name_list = []
recording_url_list = []
user_group_list = []
file_url_tail_list = []
s3_list = []

# Send request for each link in links_list_unique
for link_sub in links_list_unique:
    r = requests.get(link_sub)
    soup = BeautifulSoup(r.content, 'lxml')
    # Localize data in "userRecording" object

    try:
        counter = counter + 1
        
        # Extract information from html code:
        view = int(soup.find('li', attrs = {'class':'rec-views'}).text)
        like = soup.find('li', attrs = {'class':'rec-likes'}).text
        comment = int(soup.find('li', attrs = {'class':'rec-comments'}).text)
        recording = int(soup.find('li', attrs = {'class':'tam1'}).find('strong').text)
        follower = int(soup.find_all('ul', attrs = {'class':'stats'})[0].find_all('li')[5].find('strong').text)
        following = (soup.find_all('ul', attrs = {'class':'stats'})[0].find_all('li')[6].find('strong').text)
        date = soup.find_all('div', attrs = {'class':'infoTema_textoBlanco'})[2].text[3:]
        gender_loc = soup.find('div', attrs = {'class':'userdata'}).find('p').text

        recording_url = link_sub

        pattern = re.compile(r"userRecording")
        data = soup.find('script', text=pattern)

        string_head = 'userRecording.rid='
        # Take the tail which contains the information needed
        full_string = re.search(string_head + '.*"',str(data)).group(0)
        recording_id = full_string[len(string_head)+1:-1]         

        string_head = 'userRecording.u='
        # Take the tail which contains the information needed
        full_string = re.search(string_head + '.*"',str(data)).group(0)
        user_name = full_string[len(string_head)+1:-1] 
        
        string_head = 'userRecording.ugroup='
        # Take the tail which contains the information needed
        full_string = re.search(string_head + '.*"',str(data)).group(0)
        user_group = full_string[len(string_head)+1:-1] 

        string_head = 'userRecording.file='
        # Take the tail which contains the information needed
        full_string = re.search(string_head + '.*"',str(data)).group(0)
        file_url_tail = full_string[len(string_head)+1:-1] 

#         string_head = 'userRecording.s3='
#         # Take the tail which contains the information needed
#         full_string = re.search(string_head + '.*"',str(data)).group(0)
#         s3 = full_string[len(string_head)+1:-1] 
    except (IndexError, AttributeError) as e:
        continue
        
    view_list.append(view)
    like_list.append(like)
    comment_list.append(comment)
    recording_list.append(recording)
    follower_list.append(follower)
    following_list.append(following)
    date_list.append(date)
    gender_loc_list.append(gender_loc)
    
    recording_url_list.append(recording_url)
    recording_id_list.append(recording_id)
    user_name_list.append(user_name)
    user_group_list.append(user_group)
    file_url_tail_list.append(file_url_tail)
#     s3_list.append(s3)

    print(counter)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122


In [ ]:
file_url = []
for link_tail in file_url_tail_list:
    if link_tail[-1] == '3':
        link_full = mp3_aws_head + link_tail
    elif link_tail[-1] == '4':
        link_full = mp4_aws_head + link_tail
    file_url.append(link_full)

In [65]:
d = {'recording_id': recording_id_list, 'user_name': user_name_list, 'url_recording': recording_url_list,
     'user_group': user_group_list, 'url_file_tail': file_url_tail_list, 'no_of_views': view_list, 
     'no_of_likes': like_list, 'no_of_comments': comment_list, 'no_of_recordings': recording_list,
     'no_of_followers': follower_list, 'no_of_following': following_list, 'date_recorded': date_list,
     'gender + location': gender_loc_list, 'url_file': file_url}

In [66]:
df = pd.DataFrame(data=d)

In [67]:
df

,date_recorded,gender + location,no_of_comments,no_of_followers,no_of_following,no_of_likes,no_of_recordings,no_of_views,recording_id,url_file,url_file_tail,url_recording,user_group,user_name
0,08/07/2009,"Woman, USA, Japan",25,52,35,12,92,680,11038,https://s3.amazonaws.com/redkaraoke/data/mp3/1...,11827/8cdcce5e9807c1ad6d6c11a014c7c5a4_1.mp3,https://www.redkaraoke.com/profile/Jewel_Emi/r...,9,Jewel_Emi
1,01/12/2010,"Woman, Pozoblanco (Córdoba), Spain",15,44,25,16,30,286,624790,https://s3.amazonaws.com/redkaraoke/data/mp3/1...,11827/286d52526147e2f3df99e9c612298602_1.mp3,https://www.redkaraoke.com/profile/Bea18/recor...,2,Bea18
2,08/21/2017,Nepal,41,18,18,30,3,470,1180893,https://s3.amazonaws.com/redkaraokecom/data/vi...,11827/4e97f485ae21e97aeb64a345dee593b2_1.mp4,https://www.redkaraoke.com/profile/lucy_93/rec...,2,lucy_93
3,05/25/2017,"Woman, Kuwait",89,24,34,40,10,1434,1105848,https://s3.amazonaws.com/redkaraokecom/data/vi...,11827/63b48cbd846a8f93c7e8887a38716245_1.mp4,https://www.redkaraoke.com/profile/bhabycunana...,2,bhabycunanan
4,06/21/2016,"Woman, san felipe, Chile",17,20,20,19,3,138,2033360,https://s3.amazonaws.com/redkaraokecom/data/vi...,11827/439dd5db0434ccc409ce6c478a1caf09_1.mp4,https://www.redkaraoke.com/profile/natihytha/r...,2,natihytha
5,10/23/2016,"Woman, Székesfehérvár, Hungary",19,95,226,11,47,273,899522,https://s3.amazonaws.com/redkaraokecom/data/vi...,11827/4c743766e8efbeaff1e7c16e64ee104d_1.mp4,https://www.redkaraoke.com/profile/monikahorva...,2,monikahorvath
6,01/22/2017,Philippines,9,20,0,17,3,293,986313,https://s3.amazonaws.com/redkaraokecom/data/vi...,11827/2bb012ac0ecc1bfe791e92424a957732_1.mp4,https://www.redkaraoke.com/profile/jamzvilla/r...,2,jamzvilla
7,07/26/2017,Spain,40,18,6,38,9,758,1159769,https://s3.amazonaws.com/redkaraokecom/data/vi...,11827/27316c7ada6d21c0986a88d5eee10733_1.mp4,https://www.redkaraoke.com/profile/guadalupesa...,2,guadalupesalinas
8,07/13/2014,"Woman, In South Florida, USA",1,3839,82,55,3686,114,342153,https://s3.amazonaws.com/redkaraoke/data/mp3/1...,11827/205865feeaa7a1dbdf69e1178608a570_1.mp3,https://www.redkaraoke.com/profile/n1angel4u/r...,9,n1angel4u
9,07/05/2010,"Woman, Malaga, Spain",14,1,0,12,9,262,780489,https://s3.amazonaws.com/redkaraoke/data/mp3/1...,11827/7eea98e744176b93fe499189704fe099_1.mp3,https://www.redkaraoke.com/profile/EvaRuiz/rec...,2,EvaRuiz


In [71]:
df.to_csv('./data/out.csv',index=False)

In [53]:
# link_sub = links_list[0]
# r = requests.get(link_sub)
# soup = BeautifulSoup(r.content, 'lxml')
# soup

In [63]:
file_url

['https://s3.amazonaws.com/redkaraoke/data/mp3/11827/8cdcce5e9807c1ad6d6c11a014c7c5a4_1.mp3',
 'https://s3.amazonaws.com/redkaraoke/data/mp3/11827/286d52526147e2f3df99e9c612298602_1.mp3',
 'https://s3.amazonaws.com/redkaraokecom/data/vid/11827/4e97f485ae21e97aeb64a345dee593b2_1.mp4',
 'https://s3.amazonaws.com/redkaraokecom/data/vid/11827/63b48cbd846a8f93c7e8887a38716245_1.mp4',
 'https://s3.amazonaws.com/redkaraokecom/data/vid/11827/439dd5db0434ccc409ce6c478a1caf09_1.mp4',
 'https://s3.amazonaws.com/redkaraokecom/data/vid/11827/4c743766e8efbeaff1e7c16e64ee104d_1.mp4',
 'https://s3.amazonaws.com/redkaraokecom/data/vid/11827/2bb012ac0ecc1bfe791e92424a957732_1.mp4',
 'https://s3.amazonaws.com/redkaraokecom/data/vid/11827/27316c7ada6d21c0986a88d5eee10733_1.mp4',
 'https://s3.amazonaws.com/redkaraoke/data/mp3/11827/205865feeaa7a1dbdf69e1178608a570_1.mp3',
 'https://s3.amazonaws.com/redkaraoke/data/mp3/11827/7eea98e744176b93fe499189704fe099_1.mp3',
 'https://s3.amazonaws.com/redkaraokecom/d